## Basics of LLamaIndex

> https://gpt-index.readthedocs.io/en/stable/getting_started/starter_example.html

---

In [6]:
import sys
sys.path.append("/Users/shaunaksen/Documents/personal-projects/Natural-Language-Processing/LLM Concepts/mini_projects/chatgpt_clone")

In [27]:
import logging
import os
from llama_index.llms import AzureOpenAI
from llama_index.embeddings import AzureOpenAIEmbedding
from llama_index import (
    VectorStoreIndex, SimpleDirectoryReader, ServiceContext,
    set_global_service_context, StorageContext, load_index_from_storage
)
from creds import AZURE_API_BASE, AZURE_API_KEY, AZURE_API_VERSION

In [19]:
logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [8]:
llm = AzureOpenAI(
    deployment_name='gpt-4-32k',
    model='gpt-4-32k',
    api_key=AZURE_API_KEY,
    azure_endpoint=AZURE_API_BASE,
    api_version=AZURE_API_VERSION,
)

# You need to deploy your own embedding model as well as your own chat completion model
embed_model = AzureOpenAIEmbedding(
    model="text-embedding-ada-002",
    deployment_name="text-embedding-ada-002",
    api_key=AZURE_API_KEY,
    azure_endpoint=AZURE_API_BASE,
    api_version=AZURE_API_VERSION,
)

In [9]:
service_context = ServiceContext.from_defaults(
    llm=llm,
    embed_model=embed_model,
)

set_global_service_context(service_context)

In [10]:
# non-streaming
resp = llm.complete("Paul Graham is ")
print(resp)

an English-born computer scientist, entrepreneur, venture capitalist, author, and essayist. He is best known as the co-founder of the influential startup accelerator and seed capital firm Y Combinator, which has helped launch and fund numerous successful technology companies, including Dropbox, Airbnb, and Reddit.

Graham was born in Weymouth, England, in 1964, and moved to the United States with his family when he was a child. He earned a Bachelor of Arts in Philosophy from Cornell University, a Master of Science in Applied Mathematics from Harvard University, and a Ph.D. in Computer Science from Harvard University.

Before founding Y Combinator, Graham co-founded Viaweb, an early e-commerce platform that was acquired by Yahoo! in 1998 and became Yahoo! Store. He has also written several programming books, including "ANSI Common Lisp" and "On Lisp," and is known for his essays on technology, startups, and culture, which are published on his website, paulgraham.com.

In addition to his

In [31]:
documents = SimpleDirectoryReader(input_dir="./data/").load_data(show_progress=True)

DEBUG:llama_index.readers.file.base:> [SimpleDirectoryReader] Total files added: 1
> [SimpleDirectoryReader] Total files added: 1


Loading files: 100%|██████████| 1/1 [00:00<00:00, 388.94file/s]


In [14]:
documents[0].metadata

{'file_path': '/Users/shaunaksen/Documents/personal-projects/Natural-Language-Processing/LLM Concepts/llamaindex_tutorials/data/paul_graham_essay.txt',
 'file_name': 'paul_graham_essay.txt',
 'file_type': 'text/plain',
 'file_size': 75042,
 'creation_date': '2023-11-24',
 'last_modified_date': '2023-11-24',
 'last_accessed_date': '2023-11-24'}

In [15]:
index = VectorStoreIndex.from_documents(documents=documents)

This builds an index over the documents in the `data`` folder 
(which in this case just consists of the essay text, but could contain many documents).

In [ ]:
query_engine = index.as_query_engine()
response = query_engine.query("What did the author do growing up?")

In [17]:
print (response)

Growing up, the author mainly worked on writing and programming outside of school. They wrote short stories and experimented with programming on the IBM 1401 using an early version of Fortran. Later, they started programming on a TRS-80, creating simple games, a program to predict model rocket flight, and a word processor.


By default, the data you just loaded is stored in memory as a series of vector embeddings. You can save time (and requests to OpenAI) by saving the embeddings to disk. That can be done with this line:

In [21]:
index.storage_context.persist()

DEBUG:fsspec.local:open file: /Users/shaunaksen/Documents/personal-projects/Natural-Language-Processing/LLM Concepts/llamaindex_tutorials/storage/docstore.json
open file: /Users/shaunaksen/Documents/personal-projects/Natural-Language-Processing/LLM Concepts/llamaindex_tutorials/storage/docstore.json
DEBUG:fsspec.local:open file: /Users/shaunaksen/Documents/personal-projects/Natural-Language-Processing/LLM Concepts/llamaindex_tutorials/storage/index_store.json
open file: /Users/shaunaksen/Documents/personal-projects/Natural-Language-Processing/LLM Concepts/llamaindex_tutorials/storage/index_store.json
DEBUG:fsspec.local:open file: /Users/shaunaksen/Documents/personal-projects/Natural-Language-Processing/LLM Concepts/llamaindex_tutorials/storage/graph_store.json
open file: /Users/shaunaksen/Documents/personal-projects/Natural-Language-Processing/LLM Concepts/llamaindex_tutorials/storage/graph_store.json
DEBUG:fsspec.local:open file: /Users/shaunaksen/Documents/personal-projects/Natural-L

Of course, you don’t get the benefits of persisting unless you load the data.

In [29]:
if not os.path.exists("storage"):
    # load the documents and create the index
    documents = SimpleDirectoryReader("data").load_data()
    index = VectorStoreIndex.from_documents(documents)
    # store it for later
    index.storage_context.persist()

else:
    # load the existing index
    storage_context = StorageContext.from_defaults(persist_dir="./storage")
    index = load_index_from_storage(storage_context)

DEBUG:llama_index.storage.kvstore.simple_kvstore:Loading llama_index.storage.kvstore.simple_kvstore from ./storage/docstore.json.
Loading llama_index.storage.kvstore.simple_kvstore from ./storage/docstore.json.
DEBUG:fsspec.local:open file: /Users/shaunaksen/Documents/personal-projects/Natural-Language-Processing/LLM Concepts/llamaindex_tutorials/storage/docstore.json
open file: /Users/shaunaksen/Documents/personal-projects/Natural-Language-Processing/LLM Concepts/llamaindex_tutorials/storage/docstore.json
DEBUG:llama_index.storage.kvstore.simple_kvstore:Loading llama_index.storage.kvstore.simple_kvstore from ./storage/index_store.json.
Loading llama_index.storage.kvstore.simple_kvstore from ./storage/index_store.json.
DEBUG:fsspec.local:open file: /Users/shaunaksen/Documents/personal-projects/Natural-Language-Processing/LLM Concepts/llamaindex_tutorials/storage/index_store.json
open file: /Users/shaunaksen/Documents/personal-projects/Natural-Language-Processing/LLM Concepts/llamaindex

In [ ]:
# either way we can now query the index
query_engine = index.as_query_engine()
response = query_engine.query("What did the author do growing up?")
print(response)